In [1]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [3]:
#importing Libraries
import pandas as pd
import numpy as np
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import folium


In [43]:
pip install geocoder

     |████████████████████████████████| 98 kB 667 kB/s eta 0:00:011
Note: you may need to restart the kernel to use updated packages.


In [44]:
import geocoder

In [21]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(link,header=0)[0]
df

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [22]:
# Removing rows with df.borough == not assigned
# Resetting the index after the deletion

df_can = df[df.Borough != 'Not assigned']
df_can = df_can.reset_index(drop=True)
df_can.shape

(103, 3)

In [23]:
df_can.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


To Check the counts of each postal code 

In [25]:
count = pd.DataFrame(df_can['Postal Code'].value_counts())

In [26]:
count 

,Postal Code
M5B,1
M4L,1
M6R,1
M9L,1
M3N,1
...,...
M8Y,1
M2L,1
M6A,1
M4A,1


In [27]:
count.rename(columns={'Postal Code':'Count'},inplace=True)
count.head()

,Count
M5B,1
M4L,1
M6R,1
M9L,1
M3N,1


In [28]:
for x in count['Count']:
    if x == 2:
        print(True)

From the above test in dataframe 'Count' we confirm that there are no duplicate postal codes in dataframe df_can 

In [36]:
# Checking if col 'Neighborhood' has any value == NOT ASSIGNED
for value in df_can['Neighborhood']:
    if value =='Not assigned':
        print(value)
    

Another way to find a particular value in  a dataframe

In [39]:
df_can.loc[df_can['Neighborhood']=='Not assigned']

,Postal Code,Borough,Neighborhood


In [38]:
count.loc[count['Count']==2]

,Count


In [40]:
# Printing out the shape of the final dataframe 
df_can.shape

(103, 3)

Getting the latitude and longitude of each postal code 

In [56]:
#creating a new df of coordinates 
path= '/home/simran/Downloads/Geospatial_Coordinates.csv'

coord = pd.read_csv(path)
coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [57]:
# Merging the 2 dataframes 
df_can = df_can.merge(coord, on = 'Postal Code')
df_can.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Checking if any null value exists

In [61]:
df_can['Latitude'].isnull().value_counts()

False    103
Name: Latitude, dtype: int64

In [62]:
df_can['Longitude'].isnull().value_counts()

False    103
Name: Longitude, dtype: int64

In [65]:
df_can.shape

(103, 5)